# Model Fit in Linear Regression - Lab

## Introduction
In this lab, you'll learn how to evaluate your model results and you'll learn how to select the appropriate features using stepwise selection.

## Objectives
You will be able to:
* Use stepwise selection methods to determine the most important features for a model
* Use recursive feature elimination to determine the most important features for a model

## The Ames Housing Data once more

In [30]:
import pandas as pd
import numpy as np

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)
preprocessed.rename(columns={'1stFlrSF_log': 'FirstFlrSF_log'}, inplace=True)

In [31]:
preprocessed.head()

,LotArea_log,FirstFlrSF_log,GrLivArea_log,SalePrice_log,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,KitchenQual_Fa,KitchenQual_Gd,...,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
0,-0.133185,-0.803295,0.529078,0.559876,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0.113403,0.418442,-0.381715,0.212692,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0.419917,-0.576363,0.659449,0.733795,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.103311,-0.439137,0.541326,-0.437232,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0.878108,0.112229,1.281751,1.014303,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0


## Perform stepwise selection

The function for stepwise selection is copied below. Use this provided function on your preprocessed Ames Housing data.

In [32]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ 
    Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.pop(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [33]:
X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

result = stepwise_selection(X, y, verbose = True)
print('resulting features:')
print(result)

/Users/maxsteele/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


Add  GrLivArea_log                  with p-value 1.59847e-243
Add  KitchenQual_TA                 with p-value 1.56401e-67
Add  FirstFlrSF_log                 with p-value 7.00069e-48
Add  KitchenQual_Fa                 with p-value 1.70471e-37
Add  Neighborhood_OldTown           with p-value 3.20105e-23
Add  KitchenQual_Gd                 with p-value 4.12635e-21
Add  Neighborhood_Edwards           with p-value 9.05184e-17
Add  Neighborhood_IDOTRR            with p-value 1.10068e-18
Add  LotArea_log                    with p-value 1.71728e-13
Add  Neighborhood_NridgHt           with p-value 7.05633e-12
Add  BldgType_Duplex                with p-value 4.30647e-11
Add  Neighborhood_NAmes             with p-value 2.25803e-09
Add  Neighborhood_SWISU             with p-value 5.40743e-09
Add  Neighborhood_BrkSide           with p-value 8.79638e-10
Add  Neighborhood_Sawyer            with p-value 6.92011e-09
Add  Neighborhood_NoRidge           with p-value 5.87105e-08
Add  Neighborhood_Somer

### Build the final model again in Statsmodels

In [36]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [37]:
X_fin = X[result]
target = 'SalePrice_log'
formula = target + '~' + '+'.join(X_fin)
model = ols(formula, preprocessed).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          SalePrice_log   R-squared:                       0.835
Model:                            OLS   Adj. R-squared:                  0.832
Method:                 Least Squares   F-statistic:                     269.0
Date:                Mon, 24 Aug 2020   Prob (F-statistic):               0.00
Time:                        12:02:44   Log-Likelihood:                -754.40
No. Observations:                1460   AIC:                             1565.
Df Residuals:                    1432   BIC:                             1713.
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.2174      0.164     -1.323      0.186      -0.540       0.105
GrLivArea_log            0.3694      0.015     24.477      0.000       0.340       0.399
KitchenQual_TA          -0.7020      0.055    -12.859      0.000      -0.809      -0.595
FirstFlrSF_log           0.1445      0.015      9.645      0.000       0.115       0.174
KitchenQual_Fa          -1.0372      0.087    -11.864      0.000      -1.209      -0.866
Neighborhood_OldTown    -0.8625      0.063    -13.615      0.000      -0.987      -0.738
KitchenQual_Gd          -0.4021      0.050     -8.046      0.000      -0.500      -0.304
Neighborhood_Edwards    -0.7019      0.048    -14.530      0.000      -0.797      -0.607
Neighborhood_IDOTRR     -0.8583      0.097     -8.855      0.000      -1.048      -0.668
LotArea_log              0.1096      0.015      7.387      0.000       0.081       0.139
Neighborhood_NridgHt     0.3854      0.057      6.809      0.000       0.274       0.496
BldgType_Duplex         -0.4073      0.061     -6.678      0.000      -0.527      -0.288
Neighborhood_NAmes      -0.3763      0.038     -9.981      0.000      -0.450      -0.302
Neighborhood_SWISU      -0.6263      0.089     -7.020      0.000      -0.801      -0.451
Neighborhood_BrkSide    -0.5641      0.066     -8.493      0.000      -0.694      -0.434
Neighborhood_Sawyer     -0.4026      0.055     -7.342      0.000      -0.510      -0.295
Neighborhood_NoRidge     0.4347      0.070      6.221      0.000       0.298       0.572
Neighborhood_StoneBr     0.4538      0.087      5.226      0.000       0.283       0.624
Neighborhood_MeadowV    -0.6622      0.118     -5.592      0.000      -0.895      -0.430
SaleType_New             0.1483      0.044      3.388      0.001       0.062       0.234
Neighborhood_BrDale     -0.4733      0.123     -3.839      0.000      -0.715      -0.231
MSZoning_RM              1.0820      0.147      7.363      0.000       0.794       1.370
MSZoning_RL              0.9916      0.156      6.356      0.000       0.686       1.298
MSZoning_FV              1.2052      0.165      7.284      0.000       0.881       1.530
MSZoning_RH              0.8503      0.189      4.490      0.000       0.479       1.222
Neighborhood_NWAmes     -0.2055      0.054     -3.837      0.000      -0.311      -0.100
Neighborhood_Mitchel    -0.1943      0.065     -3.004      0.003      -0.321      -0.067
Neighborhood_SawyerW    -0.1666      0.058     -2.862      0.004      -0.281      -0.052
==============================================================================
Omnibus:                      295.535   Durbin-Watson:                   1.965
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1270.571
Skew:                          -0.903   Prob(JB):                    1.26e-276
Kurtosis

## Use Feature ranking with recursive feature elimination

Use feature ranking to select the 5 most important features

In [38]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select=5)
selector = selector.fit(X, y.values.ravel())

In [39]:
selector.support_

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False])

In [40]:
selector.ranking_

array([33, 26, 15, 30, 16, 34, 38,  2,  4,  3, 23, 19, 24, 40, 42, 22, 31,
       25,  1,  1,  1,  1, 13, 39, 10, 17, 32, 37, 36,  8, 35, 11,  7, 29,
       21, 41, 27,  1,  6, 12, 14, 20, 28, 18,  5, 43,  9])

In [41]:
selected_5 = list(selector.support_)
selected_5

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [42]:
# just for practice
for bool, pred in zip(selected_5, X.columns):
    if bool:
        print(pred)

MSZoning_FV
MSZoning_RH
MSZoning_RL
MSZoning_RM
Neighborhood_NoRidge


Fit the linear regression model again using the 5 selected columns

In [47]:
selected_predictors = X.columns[selector.support_]
linreg.fit(X[selected_predictors], y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Now, predict $\hat y$ using your model. You can use `.predict()` in scikit-learn. 

In [51]:
y_hat = linreg.predict(X[selected_predictors])

Now, using the formulas of R-squared and adjusted R-squared below, and your Python/numpy knowledge, compute them and contrast them with the R-squared and adjusted R-squared in your statsmodels output using stepwise selection. Which of the two models would you prefer?

$SS_{residual} = \sum (y - \hat{y})^2 $

$SS_{total} = \sum (y - \bar{y})^2 $

$R^2 = 1- \dfrac{SS_{residual}}{SS_{total}}$

$R^2_{adj}= 1-(1-R^2)\dfrac{n-1}{n-p-1}$

In [62]:
X[selected_predictors].columns.shape

(5,)

In [63]:
SS_resid = np.sum((y - y_hat)**2)

SS_total = np.sum((y - np.mean(y))**2)

R_squared = 1 - (SS_resid/float(SS_total))

adj_R_sq = 1 - (1 - R_squared) * ((len(y) - 1) / (len(y) - len(list(X[selected_predictors].columns)) - 1))

In [64]:
R_squared
# r_squared is 0.239434  

0.23943418177114228

In [65]:
adj_R_sq
# adjusted_r_squared is 0.236818

0.2368187559863113

## Level up (Optional)

- Perform variable selection using forward selection, using this resource: https://planspace.org/20150423-forward_selection_with_statsmodels/. Note that this time features are added based on the adjusted R-squared!
- Tweak the code in the `stepwise_selection()` function written above to just perform forward selection based on the p-value 

## Summary
Great! You practiced your feature selection skills by applying stepwise selection and recursive feature elimination to the Ames Housing dataset! 